# Классификация интентов 🏦

## Введение

В этом ноутбуке мы познакомимся с задачей **классификации интентов** - определения намерений пользователя по его текстовому запросу. Это одна из ключевых задач в создании чат-ботов и голосовых ассистентов.

**Что мы изучим:**
- Загрузку датасета Banking77 с 77 различными интентами
- Классический подход: Count Vectorizer (Bag of Words) + Логистическая регрессия
- Современный подход: Sentence Transformers (семантические эмбеддинги) + Логистическая регрессия
- Визуализацию эмбеддингов с помощью t-SNE
- Сравнение двух подходов

**Датасет Banking77:**
- 13,083 запросов на английском языке
- 77 категорий интентов (activate_my_card, age_limit, apple_pay_or_google_pay и т.д.)


## 1. Импорт библиотек


In [ ]:
# Библиотеки для работы с данными
import pandas as pd
import numpy as np

# Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

# Библиотеки для машинного обучения
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE

# Библиотека для загрузки датасета
from datasets import load_dataset

# Библиотека для sentence embeddings
from sentence_transformers import SentenceTransformer

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

print("✅ Все библиотеки успешно загружены!")


In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 2. Загрузка датасета Banking77


In [ ]:
# Загружаем датасет Banking77
dataset = load_dataset("DeepPavlov/banking77")

print("✅ Датасет успешно загружен!")
print("\nСтруктура датасета:")
print(dataset)


## 3. Исследование данных


In [ ]:
# Преобразуем в pandas DataFrame для удобства
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

print("📊 Базовая статистика:")
print(f"Размер обучающей выборки: {len(train_df)}")
print(f"Размер тестовой выборки: {len(test_df)}")
print(f"Количество уникальных интентов: {train_df['label'].nunique()}")
print("\nПервые 5 примеров:")
print(train_df.head())


Визуализируем распределение классов:


In [ ]:
# Распределение классов в обучающей выборке
plt.figure(figsize=(14, 6))

label_counts = train_df['label'].value_counts().sort_index()

plt.subplot(1, 2, 1)
plt.bar(range(len(label_counts)), label_counts.values, color='steelblue', alpha=0.7)
plt.xlabel('Номер интента', fontsize=12)
plt.ylabel('Количество примеров', fontsize=12)
plt.title('Распределение интентов в обучающей выборке', fontsize=14)
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(label_counts.values, bins=20, color='coral', alpha=0.7, edgecolor='black')
plt.xlabel('Количество примеров', fontsize=12)
plt.ylabel('Количество интентов', fontsize=12)
plt.title('Гистограмма размеров классов', fontsize=14)
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Средний размер класса: {label_counts.mean():.1f}")
print(f"Минимальный размер класса: {label_counts.min()}")
print(f"Максимальный размер класса: {label_counts.max()}")


Подготовим данные для обучения моделей:


In [ ]:
# Выделяем признаки (тексты) и целевую переменную (метки классов)
X_train = train_df['utterance'].values
y_train = train_df['label'].values

X_test = test_df['utterance'].values
y_test = test_df['label'].values

print("✅ Данные подготовлены:")
print(f"   Обучающая выборка: {len(X_train)} запросов")
print(f"   Тестовая выборка: {len(X_test)} запросов")


## 4. Базовый подход: Count Vectorizer + Логистическая регрессия 📊


### Что такое Count Vectorizer (Bag of Words)?

**Count Vectorizer** преобразует текст в числовой вектор, подсчитывая количество появлений каждого слова в тексте.

**Принцип работы:**
1. Создается словарь всех уникальных слов из всех текстов
2. Каждый текст представляется как вектор длиной, равной размеру словаря
3. В векторе на позиции каждого слова стоит количество его появлений в тексте

**Пример:**
- Словарь: ["активировать", "карту", "как", "мой", "баланс", "узнать"]
- Текст 1: "Как активировать мой карту" → [1, 1, 1, 1, 0, 0]
- Текст 2: "Как узнать мой баланс" → [0, 0, 1, 1, 1, 1]

**Преимущества:**
- Простой и быстрый
- Хорошо работает на небольших данных
- Интерпретируемый (можно увидеть важность слов)

**Недостатки:**
- Игнорирует порядок слов и контекст
- Не понимает синонимы ("активировать" ≠ "включить")
- Большая размерность при большом словаре


### 4.1. Векторизация текстов


In [ ]:
count_vectorizer = # TODO
X_train_counts = None # TODO
X_test_counts = None # TODO

### 4.2. Обучение модели


In [ ]:
# Создаем и обучаем логистическую регрессию
lr_counts = LogisticRegressionCV(random_state=42)
lr_counts.fit(X_train_counts, y_train)

print("✅ Модель обучена!")


### 4.3. Оценка качества


In [ ]:
# Делаем предсказания
y_pred_counts = lr_counts.predict(X_test_counts)

# Считаем accuracy
accuracy_counts = accuracy_score(y_test, y_pred_counts)

print("📊 Результаты Count Vectorizer + Логистическая регрессия:")
print(f"Accuracy на тестовой выборке: {accuracy_counts:.4f} ({accuracy_counts*100:.2f}%)")
print(f"\nЭто означает, что модель правильно классифицировала {int(accuracy_counts*len(y_test))} из {len(y_test)} запросов")


### 4.4. Анализ ошибок


In [ ]:
# Найдем примеры неправильных предсказаний
errors_mask = y_test != y_pred_counts
errors_indices = np.where(errors_mask)[0]

print(f"🔍 Найдено {len(errors_indices)} ошибок. Рассмотрим несколько примеров:\n")

# Покажем 5 случайных ошибок
np.random.seed(42)
sample_errors = np.random.choice(errors_indices, min(5, len(errors_indices)), replace=False)

for idx in sample_errors:
    print(f"Текст: \"{X_test[idx]}\"")
    print(f"Истинный интент: {y_test[idx]}")
    print(f"Предсказанный интент: {y_pred_counts[idx]}")
    print()


## 5. Современный подход: Sentence Transformers 🤖


### Что такое Sentence Transformers?

**Sentence Transformers** - это библиотека, которая преобразует тексты в **семантические эмбеддинги** (векторные представления), которые отражают *смысл* текста, а не просто набор слов.

### Основные отличия от Count Vectorizer:

| Характеристика | Count Vectorizer | Sentence Transformers |
|----------------|------------------|----------------------|
| **Что кодирует** | Частоту слов | Смысл предложения |
| **Синонимы** | Разные векторы | Похожие векторы |
| **Порядок слов** | Не учитывает | Учитывает контекст |
| **Размер вектора** | Размер словаря (тысячи) | Фиксированный (обычно 384-768) |
| **Обучение** | Без обучения | Предобучены на больших данных |

### Как это работает?

**Count Vectorizer** просто считает слова:
- "Activate my card" → [1, 0, 1, 1, ...]
- "Enable my card" → [0, 1, 1, 1, ...]
- Векторы **разные**, хотя смысл одинаковый!

**Sentence Transformers** понимает смысл:
- "Activate my card" → [0.15, -0.32, 0.87, ...]
- "Enable my card" → [0.14, -0.31, 0.88, ...]
- Векторы **похожи**, потому что смысл одинаковый!

### Модель all-MiniLM-L6-v2

Мы будем использовать модель **all-MiniLM-L6-v2**:
- Компактная и быстрая модель (22 млн параметров)
- Создает векторы размерности 384
- Обучена на множестве задач понимания текста
- Хорошо работает для коротких текстов (как наши запросы)

### Преимущества:
- ✅ Понимает синонимы и перефразировки
- ✅ Учитывает контекст и порядок слов
- ✅ Фиксированный размер вектора (не зависит от словаря)
- ✅ Обобщает на новые слова

### Недостатки:
- ❌ Требует больше вычислительных ресурсов
- ❌ Менее интерпретируемый (сложно понять, почему модель так решила)
- ❌ Нужна предобученная модель


## 6. Визуализация эмбеддингов 📊


Чтобы лучше понять разницу между двумя подходами, давайте визуализируем эмбеддинги!

Мы используем **t-SNE** (t-distributed Stochastic Neighbor Embedding) - метод понижения размерности, который позволяет:
- Преобразовать высокоразмерные векторы в 2D пространство
- Сохранить структуру данных (похожие тексты останутся рядом)
- Наглядно увидеть, как модели группируют тексты

Для наглядности возьмем несколько интентов и посмотрим, как их распределяют оба подхода.


### 6.1. Подготовка данных для визуализации


In [ ]:
# Выберем несколько интересных интентов для визуализации
selected_labels = [0, 5, 10, 15, 20, 25]  # 6 разных интентов
n_samples_per_label = 50  # по 50 примеров из каждого

# Отфильтруем данные
mask = train_df['label'].isin(selected_labels)
viz_df = train_df[mask].groupby('label').head(n_samples_per_label)

X_viz = viz_df['utterance'].values
y_viz = viz_df['label'].values

print(f"✅ Подготовлено {len(X_viz)} примеров для визуализации")
print(f"Выбранные интенты: {selected_labels}")


### 6.2. Получение эмбеддингов


In [ ]:
# Получаем эмбеддинги Count Vectorizer (уже обучен)
X_viz_counts = count_vectorizer.transform(X_viz).toarray()

# Загружаем модель Sentence Transformers
print("🤖 Загружаем модель Sentence Transformers...")
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Модель загружена!")

# Получаем эмбеддинги Sentence Transformers
print("🔄 Создаем эмбеддинги для текстов...")
X_viz_sentences = sentence_model.encode(X_viz, show_progress_bar=True)
print("✅ Эмбеддинги созданы!")
print(f"Размерность Count Vectorizer: {X_viz_counts.shape}")
print(f"Размерность Sentence Transformers: {X_viz_sentences.shape}")


### 6.3. Применение t-SNE


In [ ]:
# Применяем t-SNE для понижения размерности до 2D
print("🔄 Применяем t-SNE к Count Vectorizer эмбеддингам...")
tsne_counts = TSNE(n_components=2, random_state=42, perplexity=30)
X_viz_counts_2d = tsne_counts.fit_transform(X_viz_counts)

print("🔄 Применяем t-SNE к Sentence Transformers эмбеддингам...")
tsne_sentences = TSNE(n_components=2, random_state=42, perplexity=30)
X_viz_sentences_2d = tsne_sentences.fit_transform(X_viz_sentences)

print("✅ t-SNE применен!")


### 6.4. Визуализация


In [ ]:
# Создаем визуализацию
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Цветовая палитра для классов
colors = plt.cm.tab10(np.linspace(0, 1, len(selected_labels)))

# График для Count Vectorizer
for i, label in enumerate(selected_labels):
    mask = y_viz == label
    axes[0].scatter(X_viz_counts_2d[mask, 0], X_viz_counts_2d[mask, 1], 
                   c=[colors[i]], label=f'Intent {label}', alpha=0.6, s=50)

axes[0].set_title('Count Vectorizer + t-SNE', fontsize=16, fontweight='bold')
axes[0].set_xlabel('t-SNE компонента 1', fontsize=12)
axes[0].set_ylabel('t-SNE компонента 2', fontsize=12)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# График для Sentence Transformers
for i, label in enumerate(selected_labels):
    mask = y_viz == label
    axes[1].scatter(X_viz_sentences_2d[mask, 0], X_viz_sentences_2d[mask, 1], 
                   c=[colors[i]], label=f'Intent {label}', alpha=0.6, s=50)

axes[1].set_title('Sentence Transformers + t-SNE', fontsize=16, fontweight='bold')
axes[1].set_xlabel('t-SNE компонента 1', fontsize=12)
axes[1].set_ylabel('t-SNE компонента 2', fontsize=12)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7. Классификация с Sentence Transformers 🎯


Теперь используем эмбеддинги Sentence Transformers для обучения классификатора.


### 7.1. Получение эмбеддингов для всех данных


In [ ]:
# Модель уже загружена выше (sentence_model)
# Получаем эмбеддинги для обучающей выборки
print("🔄 Создаем эмбеддинги для обучающей выборки...")
X_train_embeddings = sentence_model.encode(X_train, show_progress_bar=True)

# Получаем эмбеддинги для тестовой выборки
print("🔄 Создаем эмбеддинги для тестовой выборки...")
X_test_embeddings = sentence_model.encode(X_test, show_progress_bar=True)

print("✅ Эмбеддинги созданы!")
print(f"Размер обучающей матрицы: {X_train_embeddings.shape}")
print(f"Размер тестовой матрицы: {X_test_embeddings.shape}")


### 7.2. Обучение модели


In [ ]:
lr_embeddings = None # TODO init and train model

### 7.3. Оценка качества


In [ ]:
# Делаем предсказания
y_pred_embeddings = lr_embeddings.predict(X_test_embeddings)

# Считаем accuracy
accuracy_embeddings = accuracy_score(y_test, y_pred_embeddings)

print("📊 Результаты Sentence Transformers + Логистическая регрессия:")
print(f"Accuracy на тестовой выборке: {accuracy_embeddings:.4f} ({accuracy_embeddings*100:.2f}%)")
print(f"\nЭто означает, что модель правильно классифицировала {int(accuracy_embeddings*len(y_test))} из {len(y_test)} запросов")
